# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [39]:
import pandas as pd, re

r = pd.read_csv("recipes.tsv", sep="\t")[["recipe_slug","recipe_title"]]
t = pd.read_csv("recipe-tags.tsv", sep="\t")

r["rating_raw"] = 2  # default -> 0.25
H = r"(?i)\b(?:vegan|vegetarian|salad|grilled|baked|steamed|quinoa|kale|spinach|broccoli|chickpea|lentil|tofu)\b"
B = r"\bbacon\b"

# healthy -> 5, bacon (title or tags) -> 1
r.loc[r.recipe_title.str.contains(H, regex=True, na=False), "rating_raw"] = 5
r.loc[r.recipe_title.str.contains(B, case=False, na=False), "rating_raw"] = 1
bacon = set(t.loc[t.recipe_tag.str.contains(B, case=False, na=False), "recipe_slug"])
r.loc[r.recipe_slug.isin(bacon), "rating_raw"] = 1

# ensure ≥10 zeros/ones (never pick bacon for ones)
need0 = max(0, 10 - (r.rating_raw==1).sum())
if need0: r.loc[r[r.rating_raw>1].sample(need0, random_state=0).index, "rating_raw"] = 1
need1 = max(0, 10 - (r.rating_raw==5).sum())
if need1:
    pool = r[(~r.recipe_slug.isin(bacon)) & (r.rating_raw<5)]
    r.loc[pool.sample(need1, random_state=0).index, "rating_raw"] = 5

r["rating"] = (r["rating_raw"] - 1) * 0.25
r[["recipe_slug","rating"]].to_csv("ratings.tsv", sep="\t", index=False)
print("ratings.tsv saved")


ratings.tsv saved


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [40]:

import pandas as pd

ratings = pd.read_csv("ratings.tsv", sep="\t", dtype={"recipe_slug":str})
tags    = pd.read_csv("recipe-tags.tsv", sep="\t", dtype={"recipe_slug":str,"recipe_tag":str})

ratings["slug_norm"] = ratings["recipe_slug"].str.strip().str.lower()
tags["slug_norm"]    = tags["recipe_slug"].str.strip().str.lower()

all_tags = sorted(tags["recipe_tag"].astype(str).unique())


X = pd.crosstab(index=tags["slug_norm"], columns=tags["recipe_tag"]).clip(0,1)
X = X.reindex(columns=all_tags, fill_value=0)

# Alignig rows to ratings order
X = X.reindex(index=ratings["slug_norm"], fill_value=0).astype("int8")

features = pd.concat(
    [
        ratings["recipe_slug"].reset_index(drop=True),
        pd.Series(1, index=X.index, name="bias", dtype="int8").reset_index(drop=True),
        X.reset_index(drop=True),
    ],
    axis=1,
)

print("tags expected:", len(all_tags), "tags in features:", features.shape[1]-2)
features.to_csv("features.tsv", sep="\t", index=False)
print("features.tsv saved", features.shape)


tags expected: 296 tags in features: 296
features.tsv saved (100, 298)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [41]:

import numpy as np, pandas as pd

F = pd.read_csv("features.tsv", sep="\t")
R = pd.read_csv("ratings.tsv",  sep="\t")

tag_cols = sorted([c for c in F.columns if c not in ("recipe_slug","bias")])

# Build design with bias first, then tags (bias included in fit, will not be saved)
model_cols = ["bias"] + tag_cols
X = F[model_cols].to_numpy(float)
y = R["rating"].astype(float).to_numpy()

# Ridge λ=1 with unpenalized bias
lam = 1.0
p = X.shape[1]
I = np.eye(p)
I[0,0] = 0.0  # don't penalize bias

w = np.linalg.solve(X.T @ X + lam*I, X.T @ y)

tag_coefs = w[1:]  # skip bias
pd.DataFrame({"recipe_tag": tag_cols, "coefficient": tag_coefs}).to_csv(
    "model.tsv", sep="\t", index=False
)

print("model.tsv saved — rows:", len(tag_cols), "(tags only)")


model.tsv saved — rows: 296 (tags only)


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [42]:

import numpy as np, pandas as pd

F = pd.read_csv("features.tsv", sep="\t")


tag_cols = sorted([c for c in F.columns if c not in ("recipe_slug","bias")])
model_cols = ["bias"] + tag_cols

R = pd.read_csv("ratings.tsv", sep="\t")
X = F[model_cols].to_numpy(float)
y = R["rating"].astype(float).to_numpy()

lam = 1.0
p = X.shape[1]
I = np.eye(p); I[0,0] = 0.0
w = np.linalg.solve(X.T @ X + lam*I, X.T @ y)

pred = X @ w
pd.DataFrame({"recipe_slug": F["recipe_slug"], "score_estimate": pred}).to_csv(
    "estimates.tsv", sep="\t", index=False
)

print("estimates.tsv saved — rows:", len(pred))


estimates.tsv saved — rows: 100


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [43]:
import pandas as pd, numpy as np

F = pd.read_csv("features.tsv", sep="\t")
R = pd.read_csv("ratings.tsv",  sep="\t")
X = F.drop(columns=["recipe_slug"]).to_numpy(float); r = R["rating"].to_numpy(float)
lam, alpha = 1.0, 2.0
A = lam*np.eye(X.shape[1]) + X.T@X
b = X.T@r
Ainv = np.linalg.inv(A); theta = Ainv@b
ub = (X@theta) + alpha*np.sqrt(np.einsum("ij,jk,ik->i", X, Ainv, X))
pd.DataFrame({"recipe_slug": F["recipe_slug"], "score_bound": ub}).to_csv("bounds.tsv", sep="\t", index=False)
print("bounds.tsv saved")


bounds.tsv saved


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [44]:
import pandas as pd, numpy as np
F = pd.read_csv("features.tsv", sep="\t")
R = pd.read_csv("ratings.tsv",  sep="\t").set_index("recipe_slug")["rating"]
X = F.drop(columns=["recipe_slug"]).to_numpy(float); slugs = F["recipe_slug"].tolist()
lam, alpha = 1.0, 2.0
A = lam*np.eye(X.shape[1]); b = np.zeros(X.shape[1]); recs=[]
for _ in range(100):
    Ainv = np.linalg.inv(A); theta = Ainv@b
    ucb = (X@theta) + alpha*np.sqrt(np.einsum("ij,jk,ik->i", X, Ainv, X))
    i = int(np.argmax(ucb)); s = slugs[i]; r = float(R.loc[s])
    recs.append((s, float(ucb[i]), r))
    x = X[i]; A += np.outer(x,x); b += r*x
pd.DataFrame(recs, columns=["recipe_slug","score_bound","reward"]).to_csv("recommendations.tsv", sep="\t", index=False)
print("recommendations.tsv saved")


recommendations.tsv saved


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [45]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify  and check errors, and simplify code while completing this project. 


For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68d20a8d-0870-8007-bb79-080efab54c33]
These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.